In [37]:
# !pip install numpy
# !pip install numpy-stl
# !pip install tripy
# !pip install plotly

In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from stl import mesh
# from google.colab import drive
import numpy as np

In [39]:


# drive.mount('/content/drive')
# drive_path = "/content/drive/MyDrive/Research/PhD/Scripts/"
# stl_file_path = drive_path + "test_part.stl"
stl_file_path = "./squares.stl"
# stl_file_path = "./vessel_half.stl"
# stl_file_path = "./circle_split.stl"
# stl_file_path = "./vessel_quarter.stl"
# stl_file_path = "./simple_split.stl"
# stl_file_path = "./neuro_model_solid.stl"
# stl_file_path = "./neuro_model_half.stl"





mesh_data = mesh.Mesh.from_file(stl_file_path)

# Extract vertices
vertices = mesh_data.vectors.reshape((-1, 3))

# Create a 3D scatter plot
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

fig.add_trace(go.Scatter3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    mode='markers',
    marker=dict(size=2),
))

# Set layout
fig.update_layout(scene=dict(aspectmode="data"))

# Show the plot
fig.show()

In [40]:
# def on_top_surface(triangle, top_height):
#   return np.all(triangle[:,2]>top_height)

def is_vertical(triangle):
  return not np.allclose(triangle[:, 2], triangle[0, 2], atol = 1e-4)

def has_top_edge(triangle, top_height):
  return np.sum(triangle[:, 2] > top_height) == 2
  

In [41]:
print(vertices.shape)
np.set_printoptions(suppress=True, precision=2)

height_tolerance = 1 #mm

# print(vertices)
# n_scores[:, 1:-1].max(axis=1)
# print("max z height", str(vertices[:,2].max()))

# height threshold, accounting for tolerance
height_thresh = vertices[:,2].max()-height_tolerance

triangles =  np.array([vertices[0:3,:]])

# organizes vertices into triangles
for i in range(int(vertices.shape[0]/3)):
  temp_tri = np.array([vertices[3*i:3*(i+1),:]])
  triangles = np.vstack((triangles, temp_tri))


# extracts top surface triangles that have a top edge.
vertical_tris = np.array([])
for ind in range(triangles.shape[0]):
  # checks if on top surface
  if(is_vertical(triangles[ind]) and has_top_edge(triangles[ind], height_thresh)):
    if(vertical_tris.size == 0):
      vertical_tris = np.array([triangles[ind]])
      print("triangle start shape", str(vertical_tris.shape))
    else:
      vertical_tris = np.vstack((vertical_tris, [triangles[ind]]))

print(vertical_tris.shape[0], "vertical triangles on top edge")
print(vertical_tris)


(834, 3)
triangle start shape (1, 3, 3)
66 vertical triangles on top edge
[[[  1.59  23.45   1.  ]
  [  1.59  23.45   3.  ]
  [  3.93  23.19   3.  ]]

 [[ -3.93  23.19   3.  ]
  [ -6.21  22.72   0.  ]
  [ -6.23  22.71   3.  ]]

 [[ -8.48 -25.02   3.  ]
  [ -6.54 -25.63   0.  ]
  [ -6.23 -25.71   3.  ]]

 [[ -6.23 -25.71   3.  ]
  [ -4.25 -26.13   0.  ]
  [ -3.93 -26.19   3.  ]]

 [[ -1.59 -26.45   3.  ]
  [ -3.93 -26.19   3.  ]
  [ -1.59 -26.45   1.  ]]

 [[  1.59 -26.45   3.  ]
  [  2.76 -26.35   0.  ]
  [  3.93 -26.19   3.  ]]

 [[  8.48  22.02   3.  ]
  [  7.66  22.3    0.  ]
  [  6.23  22.71   3.  ]]

 [[ -8.48  22.02   3.  ]
  [-10.62  21.13   0.  ]
  [-10.66  21.12   3.  ]]

 [[-12.74 -23.01   3.  ]
  [-10.92 -23.99   0.  ]
  [-10.66 -24.11   3.  ]]

 [[-10.66 -24.11   3.  ]
  [ -8.77 -24.91   0.  ]
  [ -8.48 -25.02   3.  ]]

 [[-25.    -1.5    3.  ]
  [-24.91  -3.67   0.  ]
  [-24.89  -3.85   3.  ]]

 [[-24.56   3.18   3.  ]
  [-24.87   1.01   0.  ]
  [-24.89   0.85   3.  ]]

 [

In [42]:
def get_top_edge(triangle, top_height):
	edge = triangle[triangle[:, 2] > top_height]
	return edge

# print(get_top_edge(vertical_tris[0], height_thresh))

#extracts the surface edge from the vertical triangles
surface_lines = np.array([])

for ind in range(vertical_tris.shape[0]):
	if(surface_lines.size == 0):
		surface_lines = np.array([get_top_edge(vertical_tris[ind], height_thresh)])
		print("triangle start shape", str(surface_lines.shape))
	else:
		# print(surface_lines.shape)
		# print(get_top_edge(vertical_tris[ind], height_thresh).shape)
		# print(get_top_edge(vertical_tris[ind], height_thresh))
		surface_lines = np.vstack((surface_lines, [get_top_edge(vertical_tris[ind], height_thresh)]))

print(surface_lines.shape)
print(surface_lines)

triangle start shape (1, 2, 3)
(66, 2, 3)
[[[  1.59  23.45   3.  ]
  [  3.93  23.19   3.  ]]

 [[ -3.93  23.19   3.  ]
  [ -6.23  22.71   3.  ]]

 [[ -8.48 -25.02   3.  ]
  [ -6.23 -25.71   3.  ]]

 [[ -6.23 -25.71   3.  ]
  [ -3.93 -26.19   3.  ]]

 [[ -1.59 -26.45   3.  ]
  [ -3.93 -26.19   3.  ]]

 [[  1.59 -26.45   3.  ]
  [  3.93 -26.19   3.  ]]

 [[  8.48  22.02   3.  ]
  [  6.23  22.71   3.  ]]

 [[ -8.48  22.02   3.  ]
  [-10.66  21.12   3.  ]]

 [[-12.74 -23.01   3.  ]
  [-10.66 -24.11   3.  ]]

 [[-10.66 -24.11   3.  ]
  [ -8.48 -25.02   3.  ]]

 [[-25.    -1.5    3.  ]
  [-24.89  -3.85   3.  ]]

 [[-24.56   3.18   3.  ]
  [-24.89   0.85   3.  ]]

 [[  8.48 -25.02   3.  ]
  [ 10.66 -24.11   3.  ]]

 [[ 10.66 -24.11   3.  ]
  [ 12.74 -23.01   3.  ]]

 [[ 12.74  20.01   3.  ]
  [ 10.66  21.12   3.  ]]

 [[-12.74  20.01   3.  ]
  [-14.7   18.72   3.  ]]

 [[-16.54 -20.24   3.  ]
  [-14.7  -21.72   3.  ]]

 [[-18.23 -18.61   3.  ]
  [-16.54 -20.24   3.  ]]

 [[-19.76 -16.82   3. 

In [43]:
def shares_point(poly1,poly2):
  if(len(poly2.shape)>2):
    poly2 = np.squeeze(poly2)
  if(len(poly1.shape)>2):
    poly1 = np.squeeze(poly1)
  for i in range(poly1.shape[0]):
    for j in range(poly2.shape[0]):
      if(np.allclose(poly1[i], poly2[j],atol=0.01)):
          return True
  return False


def get_loops(lines):
	#used to track which lines are already "claimed"
	line_used = np.full(lines.shape[0], False)
	line_loops = []
	
	#until all lines have been used
	while not np.all(line_used == True):
		#initiates temporary loop for population
		temp_start_index =np.argmax(~line_used) #finds the first unclaimed line to seed next shape
		# print("new start index: " , temp_start_index)
		temp_start_line = lines[temp_start_index]
		line_used[temp_start_index] = True
		temp_loop = np.array([temp_start_line])
		temp_line = temp_start_line
		loop_closed = False
		#checking for next line in sequence
		while not loop_closed:
			for i in range(lines.shape[0]):
				#starts at last line start for speed
				wrapped_ind = (temp_start_index+i)%lines.shape[0]
				#checks if a point is shared with the last line
				if (not line_used[wrapped_ind]) and shares_point(lines[wrapped_ind], temp_line):
					# print("adding line ", wrapped_ind)
					line_used[wrapped_ind] = True
					temp_line = lines[wrapped_ind]
					temp_loop = np.vstack((temp_loop, [temp_line]))
					# print("loop dimension" , temp_loop.shape)

					#loop is closed
					#if not the starting line and shares a point with the starting line
					# if(not np.allclose(temp_line,  temp_start_line, atol=0.001 ) and shares_point(temp_line, temp_start_line)):
					if( temp_loop.shape[0]>2 and shares_point(temp_line, temp_start_line)):
							line_loops.append(temp_loop)
							# print("Loop closed! Now the numberof loops is ", len(line_loops))
							# print(line_loops[-1])
							loop_closed = True
							break
			# print("Failed to find adjacent line")
		
	return line_loops
					

loops = get_loops(surface_lines)

for i in range(len(loops)):
	print("loop ", i, ":")
	print("points: ",  loops[i].shape)
	for j in range(loops[i].shape[0]):
		print(loops[i][j])

	print("\n")
	


loop  0 :
points:  (33, 2, 3)
[[ 1.59 23.45  3.  ]
 [ 3.93 23.19  3.  ]]
[[ 6.23 22.71  3.  ]
 [ 3.93 23.19  3.  ]]
[[ 8.48 22.02  3.  ]
 [ 6.23 22.71  3.  ]]
[[10.66 21.12  3.  ]
 [ 8.48 22.02  3.  ]]
[[12.74 20.01  3.  ]
 [10.66 21.12  3.  ]]
[[14.7  18.72  3.  ]
 [12.74 20.01  3.  ]]
[[16.54 17.25  3.  ]
 [14.7  18.72  3.  ]]
[[18.23 15.61  3.  ]
 [16.54 17.25  3.  ]]
[[19.76 13.82  3.  ]
 [18.23 15.61  3.  ]]
[[21.11 11.89  3.  ]
 [19.76 13.82  3.  ]]
[[22.28  9.85  3.  ]
 [21.11 11.89  3.  ]]
[[23.25  7.7   3.  ]
 [22.28  9.85  3.  ]]
[[24.01  5.47  3.  ]
 [23.25  7.7   3.  ]]
[[24.56  3.18  3.  ]
 [24.01  5.47  3.  ]]
[[24.89  0.85  3.  ]
 [24.56  3.18  3.  ]]
[[25.   -1.5   3.  ]
 [24.89  0.85  3.  ]]
[[24.89 -3.85  3.  ]
 [25.   -1.5   3.  ]]
[[24.56 -6.18  3.  ]
 [24.89 -3.85  3.  ]]
[[24.01 -8.47  3.  ]
 [24.56 -6.18  3.  ]]
[[ 23.25 -10.7    3.  ]
 [ 24.01  -8.47   3.  ]]
[[ 22.28 -12.84   3.  ]
 [ 23.25 -10.7    3.  ]]
[[ 21.11 -14.89   3.  ]
 [ 22.28 -12.84   3.  ]]
[[ 19.

In [44]:
#ensures that the points on each line is in the correct order

def fix_loop_order(loop):
	# checks if first point is incorrectly ordered.
	if np.allclose(loop[0,0,:], loop[1,0,:],atol=0.01) or np.allclose(loop[0,0,:], loop[1,1,:],atol=0.01):
		print("Switched first from ")
		print(loop[0,:,:])
		tempA = np.copy(loop[0,0,:])
		tempB = np.copy(loop[0,1,:])

		loop[0,0,:] = tempB
		loop[0,1,:] = tempA
		print("to:  ")
		print(loop[0,:,:])
		

	for line_ind in range(loop.shape[0]-1):
		#checks if the second point in the current segment is the same as the first point in the next segment. If not, switches it
		if not np.allclose(loop[line_ind,1,:], loop[line_ind+1,0,:],atol=0.01):
			tempA = np.copy(loop[line_ind+1,0,:])
			tempB = np.copy(loop[line_ind+1,1,:])
			
			loop[line_ind+1,0,:] = tempB
			loop[line_ind+1,1,:] = tempA
	return loop


for i in range(len(loops)):
	# print("loop ", i, ":")
	# print("points: ",  loops[i].shape[0])
	loops[i] = fix_loop_order(loops[i])

print("Updated loop orders: ")
for i in range(len(loops)):
	print("loop ", i, ":")
	print("points: ",  loops[i].shape[0])
	for j in range(loops[i].shape[0]):
		print(loops[i][j])

	print("\n")


	

Updated loop orders: 
loop  0 :
points:  33
[[ 1.59 23.45  3.  ]
 [ 3.93 23.19  3.  ]]
[[ 3.93 23.19  3.  ]
 [ 6.23 22.71  3.  ]]
[[ 6.23 22.71  3.  ]
 [ 8.48 22.02  3.  ]]
[[ 8.48 22.02  3.  ]
 [10.66 21.12  3.  ]]
[[10.66 21.12  3.  ]
 [12.74 20.01  3.  ]]
[[12.74 20.01  3.  ]
 [14.7  18.72  3.  ]]
[[14.7  18.72  3.  ]
 [16.54 17.25  3.  ]]
[[16.54 17.25  3.  ]
 [18.23 15.61  3.  ]]
[[18.23 15.61  3.  ]
 [19.76 13.82  3.  ]]
[[19.76 13.82  3.  ]
 [21.11 11.89  3.  ]]
[[21.11 11.89  3.  ]
 [22.28  9.85  3.  ]]
[[22.28  9.85  3.  ]
 [23.25  7.7   3.  ]]
[[23.25  7.7   3.  ]
 [24.01  5.47  3.  ]]
[[24.01  5.47  3.  ]
 [24.56  3.18  3.  ]]
[[24.56  3.18  3.  ]
 [24.89  0.85  3.  ]]
[[24.89  0.85  3.  ]
 [25.   -1.5   3.  ]]
[[25.   -1.5   3.  ]
 [24.89 -3.85  3.  ]]
[[24.89 -3.85  3.  ]
 [24.56 -6.18  3.  ]]
[[24.56 -6.18  3.  ]
 [24.01 -8.47  3.  ]]
[[ 24.01  -8.47   3.  ]
 [ 23.25 -10.7    3.  ]]
[[ 23.25 -10.7    3.  ]
 [ 22.28 -12.84   3.  ]]
[[ 22.28 -12.84   3.  ]
 [ 21.11 -14.89  

In [45]:
# conversion to polygons defined by points (not lines anymore)

def loop_to_poly(line_loop):
	poly = line_loop[0,0,:]
	for line_ind in range(1,line_loop.shape[0]):
		poly = np.vstack((poly, line_loop[line_ind,0,:]))
	return poly

# print(loops[0].shape)
# print(loops[0])
# print("------")
# print(loop_to_poly(loops[0]))

polys = []
for i in range(len(loops)):
	polys.append(loop_to_poly(loops[i]))

print(polys)

[array([[  1.59,  23.45,   3.  ],
       [  3.93,  23.19,   3.  ],
       [  6.23,  22.71,   3.  ],
       [  8.48,  22.02,   3.  ],
       [ 10.66,  21.12,   3.  ],
       [ 12.74,  20.01,   3.  ],
       [ 14.7 ,  18.72,   3.  ],
       [ 16.54,  17.25,   3.  ],
       [ 18.23,  15.61,   3.  ],
       [ 19.76,  13.82,   3.  ],
       [ 21.11,  11.89,   3.  ],
       [ 22.28,   9.85,   3.  ],
       [ 23.25,   7.7 ,   3.  ],
       [ 24.01,   5.47,   3.  ],
       [ 24.56,   3.18,   3.  ],
       [ 24.89,   0.85,   3.  ],
       [ 25.  ,  -1.5 ,   3.  ],
       [ 24.89,  -3.85,   3.  ],
       [ 24.56,  -6.18,   3.  ],
       [ 24.01,  -8.47,   3.  ],
       [ 23.25, -10.7 ,   3.  ],
       [ 22.28, -12.84,   3.  ],
       [ 21.11, -14.89,   3.  ],
       [ 19.76, -16.82,   3.  ],
       [ 18.23, -18.61,   3.  ],
       [ 16.54, -20.24,   3.  ],
       [ 14.7 , -21.72,   3.  ],
       [ 12.74, -23.01,   3.  ],
       [ 10.66, -24.11,   3.  ],
       [  8.48, -25.02,   3.  ],
       [ 

In [46]:
#ensures that all output polygons are defined by points in clockwise order.
# based on  https://stackoverflow.com/questions/1165647/how-to-determine-if-a-list-of-polygon-points-are-in-clockwise-order#:~:text=Calculate%20the%20signed%20area%20of,it's%20positive%20they%20are%20counterclockwise.
def signedArea(x1,y1,x2,y2):
	return (x1*y2) - (x2*y1)

def is_clockwise(polygon):
	# print(polygon.shape)
	area = signedArea(polygon[-1,0], polygon[-1,1], polygon[0,0], polygon[0,1])
	for i in range(polygon.shape[0]-1):
		area = area+signedArea(polygon[i,0], polygon[i,1], polygon[i+1,0], polygon[i+1,1])
		#  If it's negative = clockwise order,  positive = counterclockwise.
	if area < 0:
		return True
	return False



	
for i in range(len(polys)):
	# ccw
	if not is_clockwise(polys[i]):
		print("flipping from : ")
		print(polys[i])
		polys[i] = np.flip(polys[i],0)
		print("to: ")
		print(polys[i])
		print("\n")
		
		

flipping from : 
[[ -3.93  23.19   3.  ]
 [ -6.23  22.71   3.  ]
 [ -8.48  22.02   3.  ]
 [-10.66  21.12   3.  ]
 [-12.74  20.01   3.  ]
 [-14.7   18.72   3.  ]
 [-16.54  17.25   3.  ]
 [-18.23  15.61   3.  ]
 [-19.76  13.82   3.  ]
 [-21.11  11.89   3.  ]
 [-22.28   9.85   3.  ]
 [-23.25   7.7    3.  ]
 [-24.01   5.47   3.  ]
 [-24.56   3.18   3.  ]
 [-24.89   0.85   3.  ]
 [-25.    -1.5    3.  ]
 [-24.89  -3.85   3.  ]
 [-24.56  -6.18   3.  ]
 [-24.01  -8.47   3.  ]
 [-23.25 -10.7    3.  ]
 [-22.28 -12.84   3.  ]
 [-21.11 -14.89   3.  ]
 [-19.76 -16.82   3.  ]
 [-18.23 -18.61   3.  ]
 [-16.54 -20.24   3.  ]
 [-14.7  -21.72   3.  ]
 [-12.74 -23.01   3.  ]
 [-10.66 -24.11   3.  ]
 [ -8.48 -25.02   3.  ]
 [ -6.23 -25.71   3.  ]
 [ -3.93 -26.19   3.  ]
 [ -1.59 -26.45   3.  ]
 [ -1.59  23.45   3.  ]]
to: 
[[ -1.59  23.45   3.  ]
 [ -1.59 -26.45   3.  ]
 [ -3.93 -26.19   3.  ]
 [ -6.23 -25.71   3.  ]
 [ -8.48 -25.02   3.  ]
 [-10.66 -24.11   3.  ]
 [-12.74 -23.01   3.  ]
 [-14.7  -21.72  

In [47]:
# moving everything to the first quadrant.

def reset_origin_polys(polygons, x_offset=0.0, y_offset=0.0):
	min_x =  float("inf")
	min_y =  float("inf")
	# getting minimum indices
	for poly_ind in range(int(len(polygons))):
		# if(surface_tris[tri_ind].shape[0] != 3):
		# 	print("Shape is not a triangle! ", str(surface_tris[tri_ind].shape[0]), " Points detected!")
		# 	print(surface_tris[tri_ind])
		for point_ind in range(polygons[poly_ind].shape[0]):
			if polygons[poly_ind][point_ind,0] < min_x:
				min_x = polygons[poly_ind][point_ind,0] 
			if polygons[poly_ind][point_ind,1] < min_y:
				min_y = polygons[poly_ind][point_ind,1] 
	
	print()
	# updating all points
	for poly_ind in range(int(len(polygons))):

		
		for point_ind in range(polygons[poly_ind].shape[0]):
			if(min_x<0):
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]+abs(min_x)+x_offset
			else:
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]-abs(min_x)+x_offset

			if(min_y<0):
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]+abs(min_y)+y_offset
			else:
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]-abs(min_y)+y_offset


	return polygons

print(polys)
polygons = reset_origin_polys(polys,45,15)
print(polygons)


[array([[  1.59,  23.45,   3.  ],
       [  3.93,  23.19,   3.  ],
       [  6.23,  22.71,   3.  ],
       [  8.48,  22.02,   3.  ],
       [ 10.66,  21.12,   3.  ],
       [ 12.74,  20.01,   3.  ],
       [ 14.7 ,  18.72,   3.  ],
       [ 16.54,  17.25,   3.  ],
       [ 18.23,  15.61,   3.  ],
       [ 19.76,  13.82,   3.  ],
       [ 21.11,  11.89,   3.  ],
       [ 22.28,   9.85,   3.  ],
       [ 23.25,   7.7 ,   3.  ],
       [ 24.01,   5.47,   3.  ],
       [ 24.56,   3.18,   3.  ],
       [ 24.89,   0.85,   3.  ],
       [ 25.  ,  -1.5 ,   3.  ],
       [ 24.89,  -3.85,   3.  ],
       [ 24.56,  -6.18,   3.  ],
       [ 24.01,  -8.47,   3.  ],
       [ 23.25, -10.7 ,   3.  ],
       [ 22.28, -12.84,   3.  ],
       [ 21.11, -14.89,   3.  ],
       [ 19.76, -16.82,   3.  ],
       [ 18.23, -18.61,   3.  ],
       [ 16.54, -20.24,   3.  ],
       [ 14.7 , -21.72,   3.  ],
       [ 12.74, -23.01,   3.  ],
       [ 10.66, -24.11,   3.  ],
       [  8.48, -25.02,   3.  ],
       [ 

In [48]:
# exporting to a file

num_polys = len(polygons)
# output_name = "neuro_model_poly.txt"
output_name = "squares.txt"
# output_name = "circle_split.txt"
# output_name = "simple_split.txt"
# output_name = "neuro_model_half.txt"


with open(output_name, "wb") as f:
	f.write(str("num polygons\n").encode())
	f.write(str(str(num_polys) + "\n\n").encode())
	for poly_ind in range(int(num_polys)):
		f.write(str(polygons[poly_ind].shape[0]).encode())
		f.write(("\n").encode())
		for point_ind in range(polygons[poly_ind].shape[0]):
			np.savetxt(f,[polygons[poly_ind][point_ind,0:2]], fmt='%.2f',delimiter=',')
		f.write(b"\n")


-----------------------